In [1]:
# ============================================================
# EEGConformer Masked Reconstruction (SSL)
# - 129ch 캐시 → Cz 제거(128ch)
# - Block masking (연속 구간 가림)
# - 2-layer Conv1d decoder
# - 복원은 (B,C,T) 전체, 손실은 '가려진 구간'만
# ============================================================

import os, random, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from braindecode.models import EEGConformer

# -----------------------------
# CONFIG
# -----------------------------
CACHE_DIR   = "/data5/open_data/HBN/cache_eeg_windows"
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ       = 100
N_CHANS     = 128           # Cz 제거 후
CROP_T      = 200           # SSL은 짧게 (데이터 수 ↑)
MASK_RATIO  = 0.30          # 전체 시간의 30%를 가림(연속 블록)
BLOCK_LEN   = 32            # 블록당 길이(샘플) — 100Hz 기준 0.32s
BATCH_SIZE  = 16
EPOCHS      = 50
LR          = 3e-4
WD          = 1e-4
SEED        = 42
NUM_WORKERS = 4
EMBED_DIM   = 256           # EEGConformer latent dim
GRAD_CLIP   = 1.0

def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed()

# ============================================================
# 1) BLOCK MASKING
# ============================================================
def make_block_mask(T, mask_ratio=0.3, block_len=32):
    """
    True=보이는 위치, False=가리는 위치 (연속 블록)
    블록이 영상처럼 겹칠 수 있지만, 비율을 대략 만족
    """
    keep = np.ones(T, dtype=bool)
    target_mask = int(T * mask_ratio)
    if target_mask <= 0:
        return keep
    # 최소 1블록
    n_blocks = max(1, target_mask // block_len)
    for _ in range(n_blocks):
        start = np.random.randint(0, max(1, T - block_len + 1))
        keep[start:start+block_len] = False
    # 필요 시 잔여 부분 랜덤 보완(조금 더 가릴 수 있음)
    masked_now = (~keep).sum()
    while masked_now < target_mask:
        idx = np.random.randint(0, T)
        if keep[idx]:
            keep[idx] = False
            masked_now += 1
    return keep

# ============================================================
# 2) DATASET
# ============================================================
class MaskedEEGDataset(Dataset):
    """
    - 캐시 파일에서 랜덤 세그먼트 하나 선택 → Cz 제거(129->128)
    - 랜덤 크롭(CROP_T), 짧으면 zero-pad
    - per-channel z-score
    - block keep-mask 생성 (True=보임, False=가림)
    """
    def __init__(self, cache_dir, crop_len=200, mask_ratio=0.3, block_len=32):
        self.files = [os.path.join(cache_dir, f) for f in os.listdir(cache_dir)
                      if f.endswith(".npy")]
        self.crop_len = crop_len
        self.mask_ratio = mask_ratio
        self.block_len = block_len

    def __len__(self): return len(self.files)

    def __getitem__(self, i):
        arr = np.load(self.files[i], mmap_mode="r")               # (segments, C, T)
        seg = arr[random.randint(0, arr.shape[0]-1)]              # (C, T)

        # ✅ Cz 제거 (129 → 128)
        if seg.shape[0] == 129:
            seg = seg[:-1, :]

        C, T = seg.shape
        # ✅ 길이 부족 시 zero-pad
        if T < self.crop_len:
            pad = np.zeros((C, self.crop_len), dtype=np.float32)
            pad[:, :T] = seg
            seg = pad
            T = self.crop_len

        # ✅ 안전한 크롭
        start = random.randint(0, T - self.crop_len) if T > self.crop_len else 0
        x = seg[:, start:start + self.crop_len]                   # (C, crop_len)

        # ✅ per-channel z-score
        x = (x - x.mean(axis=1, keepdims=True)) / (x.std(axis=1, keepdims=True) + 1e-6)

        # ✅ block keep-mask (1, T)
        keep = make_block_mask(self.crop_len, mask_ratio=self.mask_ratio, block_len=self.block_len)
        keep = torch.tensor(keep, dtype=torch.bool).unsqueeze(0)  # (1, T)

        x = torch.tensor(x, dtype=torch.float32)                  # (C, T)
        return x, keep

# ============================================================
# 3) MODEL: Encoder + 2-layer Decoder
# ============================================================
class EEGMaskedAutoencoder(nn.Module):
    """
    - Encoder: EEGConformer(n_outputs=embed_dim)
      -> (B, embed_dim, T_enc) 또는 (B, embed_dim)
    - Decoder: 2-layer Conv1d (embed_dim -> 2*embed_dim -> C)
    - 입력:  x: (B, C, T), keep: (B, 1, T)  (True=보임, False=가림)
    - 출력:  x_hat: (B, C, T), 손실은 ~keep(가려진 구간)에서만 계산
    """
    def __init__(self, n_chans=128, n_times=200, sfreq=100, embed_dim=256):
        super().__init__()
        self.encoder = EEGConformer(
            n_chans=n_chans,
            n_outputs=embed_dim,   # latent channels
            n_times=n_times,
            sfreq=sfreq
        )
        self.decoder = nn.Sequential(
            nn.Conv1d(embed_dim, embed_dim * 2, kernel_size=3, padding=1),
            nn.GELU(),
            nn.Conv1d(embed_dim * 2, n_chans, kernel_size=1)
        )

    def forward(self, x, keep_bool):
        # x: (B, C, T), keep_bool: (B, 1, T)
        B, C, T = x.shape

        # ✅ 가려진 곳 zero-out (브로드캐스팅)
        visible = x * keep_bool.float()                           # (B,C,T) * (B,1,T)

        # Encoder
        z = self.encoder(visible)                                 # (B, embed_dim, T_enc) or (B, embed_dim)
        if z.ndim == 2:
            z = z.unsqueeze(-1)                                   # (B, embed_dim, 1)

        # 🔧 시간 길이 보정 (필요 시)
        T_enc = z.shape[-1]
        if T_enc != T:
            z = F.interpolate(z, size=T, mode="linear", align_corners=False)

        # 2-layer decoder
        x_hat = self.decoder(z)                                   # (B, C, T)
        return x_hat

def masked_mse(pred, target, keep_bool, eps=1e-8):
    """
    pred/target: (B,C,T), keep_bool: (B,1,T) (True=보임)
    손실은 '가려진 구간'(~keep)만 평균
    """
    masked = (~keep_bool).float()                                 # (B,1,T)
    diff2 = (pred - target) ** 2                                  # (B,C,T)
    num = (diff2 * masked).sum()
    den = masked.sum() * pred.size(1) + eps                       # (mask된 타임수 × 채널수)
    return num / den

# ============================================================
# 4) TRAIN
# ============================================================
def main():
    ds = MaskedEEGDataset(CACHE_DIR, crop_len=CROP_T, mask_ratio=MASK_RATIO, block_len=BLOCK_LEN)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=NUM_WORKERS, pin_memory=True)

    model = EEGMaskedAutoencoder(n_chans=N_CHANS, n_times=CROP_T, sfreq=SFREQ, embed_dim=EMBED_DIM).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    sch = torch.optim.lr_scheduler.CosineAnnealingLR(opt, T_max=EPOCHS)

    for ep in range(1, EPOCHS + 1):
        model.train(); total = 0.0
        for x, keep in tqdm(loader, desc=f"SSL Masked Ep{ep}"):
            x = x.to(DEVICE, non_blocking=True)                   # (B,C,T)
            keep = keep.to(DEVICE, non_blocking=True)             # (B,1,T)

            opt.zero_grad()
            x_hat = model(x, keep)
            loss = masked_mse(x_hat, x, keep)
            loss.backward()
            if GRAD_CLIP is not None:
                nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
            opt.step()
            total += loss.item()

        sch.step()
        print(f"[Ep {ep:02d}] masked-recon MSE = {total / max(1,len(loader)):.5f}")

    # Encoder 파라미터만 저장 → downstream에서 로드
    torch.save(model.encoder.state_dict(), "pretrained_eegconformer_masked_block2dec.pth")
    print("✅ Saved: pretrained_eegconformer_masked_block2dec.pth")

if __name__ == "__main__":
    main()


/usr/lib/python3/dist-packages/jinja2/loaders.py:222: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DefaultProvider, ResourceManager, \
/home/RA/.local/lib/python3.10/site-packages/braindecode/models/eegconformer.py:227: UserWarning: This model has only been tested on no more than 64 channels. no guarantee to work with more channels.
  warnings.warn(
SSL Masked Ep1: 100%|██████████| 20/20 [00:03<00:00,  5.40it/s]


[Ep 01] masked-recon MSE = 0.97043


SSL Masked Ep2: 100%|██████████| 20/20 [00:03<00:00,  6.01it/s]


[Ep 02] masked-recon MSE = 0.96906


SSL Masked Ep3: 100%|██████████| 20/20 [00:03<00:00,  6.47it/s]


...

[Ep 48] masked-recon MSE = 0.96709


SSL Masked Ep49: 100%|██████████| 20/20 [00:01<00:00, 11.34it/s]


[Ep 49] masked-recon MSE = 0.96932


SSL Masked Ep50: 100%|██████████| 20/20 [00:02<00:00,  6.93it/s]

[Ep 50] masked-recon MSE = 0.95301
✅ Saved: pretrained_eegconformer_masked_block2dec.pth


In [6]:
# ssl_pretrain_cbramod.py
import os, random, numpy as np, torch, sys
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# CBraMod import (레포 클론한 경로에서 실행)
CBRAMOD_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod"
if CBRAMOD_PATH not in sys.path:
    sys.path.append(CBRAMOD_PATH)
from models.cbramod import CBraMod
import torch.nn as nn

# ---------------- config ----------------
CACHE_DIR   = "/data5/open_data/HBN/cache_eeg_windows"
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ       = 100
N_CHANS_IN  = 128           # 129면 Cz 제거하여 128 사용
S, P        = 4, 200        # (time_segments, points_per_patch) -> 800 samples
BATCH_SIZE  = 16
EPOCHS      = 100
LR          = 3e-4
WD          = 1e-4
MASK_RATIO  = 0.30
BLOCK_LEN   = 32            # 연속 가림 길이(샘플)
SEED        = 42

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

def make_block_mask(P_len, mask_ratio=0.3, block_len=32):
    """한 패치 길이 P에 대해 True=보임, False=가림 (연속 블록)"""
    keep = np.ones(P_len, dtype=bool)
    target = int(P_len * mask_ratio)
    n_blocks = max(1, target // block_len)
    for _ in range(n_blocks):
        st = np.random.randint(0, max(1, P_len - block_len + 1))
        keep[st:st+block_len] = False
    # 부족분 보완
    while (~keep).sum() < target:
        i = np.random.randint(0, P_len)
        keep[i] = False
    return keep  # (P,)

class HBNCBraModSSL(Dataset):
    """HBN cache → (C,S,P), keep-mask: (1,S,P)"""
    def __init__(self, cache_dir, S=4, P=200):
        self.files = [os.path.join(cache_dir, f) for f in os.listdir(cache_dir) if f.endswith(".npy")]
        self.S, self.P = S, P
        self.win_len = S * P

    def __len__(self): return len(self.files)

    def __getitem__(self, i):
        arr = np.load(self.files[i], mmap_mode="r")        # (segments, C, T)
        seg = arr[np.random.randint(0, arr.shape[0])]      # (C, T)
        # Cz 제거 (129 -> 128)
        if seg.shape[0] == 129:
            seg = seg[:-1, :]
        C, T = seg.shape
        # 길이 보정(>=win_len): 랜덤 크롭, (<win_len): zero-pad
        if T < self.win_len:
            pad = np.zeros((C, self.win_len), np.float32)
            pad[:, :T] = seg
            seg = pad
        else:
            st = np.random.randint(0, T - self.win_len + 1)
            seg = seg[:, st:st + self.win_len]

        # 표준화 (채널별)
        seg = (seg - seg.mean(axis=1, keepdims=True)) / (seg.std(axis=1, keepdims=True) + 1e-6)
        # (C, S, P)
        x = torch.tensor(seg.reshape(C, self.S, self.P), dtype=torch.float32)

        # keep-mask 만들기: 패치마다 P축 블록마스킹
        keep = np.stack([make_block_mask(self.P, MASK_RATIO, BLOCK_LEN) for _ in range(self.S)], axis=0)  # (S,P)
        keep = torch.tensor(keep, dtype=torch.bool).unsqueeze(0)  # (1, S, P)

        return x, keep

def masked_mse(pred, target, keep_bool, eps=1e-8):
    """
    pred/target: (B,C,S,P), keep_bool: (B,1,S,P)  (True=보임)
    손실은 '가려진 구간'(~keep)만 평균
    """
    masked = (~keep_bool).float()                          # (B,1,S,P)
    diff2  = (pred - target) ** 2                          # (B,C,S,P)
    num = (diff2 * masked).sum()
    den = masked.sum() * pred.size(1) + eps               # (mask된 시간수×채널수)
    return num / den

def main():
    ds = HBNCBraModSSL(CACHE_DIR, S=S, P=P)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

    # ✅ CBraMod 로드: proj_out 제거해 feature map (B,C,S,P) 받기
    model = CBraMod().to(DEVICE)
    model.proj_out = nn.Identity()   # README Quick Start 권장 방식 :contentReference[oaicite:2]{index=2}

    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

    for ep in range(1, EPOCHS+1):
        model.train(); running = 0.0
        for x, keep in tqdm(loader, desc=f"CBraMod-SSL Ep{ep}"):
            x    = x.to(DEVICE, non_blocking=True)              # (B,C,S,P)
            keep = keep.to(DEVICE, non_blocking=True)           # (B,1,S,P)
            # 입력 가리기
            visible = x * keep.float()

            opt.zero_grad()
            with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
                pred = model(visible)                           # (B,C,S,P) — Quick Start와 동일한 출력 포맷 :contentReference[oaicite:3]{index=3}
                loss = masked_mse(pred, x, keep)

            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()
            running += loss.item()

        print(f"[Ep {ep:02d}] masked MSE: {running/max(1,len(loader)):.5f}")

    # 인코더 전체 저장(그대로 로드해서 다운스트림에 사용)
    torch.save(model.state_dict(), "cbramod_ssl_hbn.pth")
    print("✅ Saved SSL checkpoint: cbramod_ssl_hbn.pth")

if __name__ == "__main__":
    main()


CBraMod-SSL Ep1: 100%|██████████| 20/20 [00:02<00:00,  8.23it/s]


[Ep 01] masked MSE: 19.43482


CBraMod-SSL Ep2: 100%|██████████| 20/20 [00:02<00:00,  7.56it/s]


[Ep 02] masked MSE: 7.34289


CBraMod-SSL Ep3: 100%|██████████| 20/20 [00:02<00:00,  8.03it/s]


...

[Ep 98] masked MSE: 0.62267


CBraMod-SSL Ep99: 100%|██████████| 20/20 [00:03<00:00,  6.47it/s]


[Ep 99] masked MSE: 0.61323


CBraMod-SSL Ep100: 100%|██████████| 20/20 [00:03<00:00,  5.71it/s]


[Ep 100] masked MSE: 0.62279
✅ Saved SSL checkpoint: cbramod_ssl_hbn.pth


In [7]:
# ssl_finetune_hbn_from_pretrained.py
import os, random, numpy as np, torch, sys
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ========== CBraMod import ==========
CBRAMOD_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod"
if CBRAMOD_PATH not in sys.path:
    sys.path.append(CBRAMOD_PATH)
from models.cbramod import CBraMod
import torch.nn as nn

# ========== Config ==========
CACHE_DIR   = "/data5/open_data/HBN/cache_eeg_windows"
DEVICE      = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ       = 100
N_CHANS     = 128
S, P        = 4, 200
MASK_RATIO  = 0.3
BLOCK_LEN   = 32
BATCH_SIZE  = 16
EPOCHS      = 50
LR          = 3e-4
WD          = 1e-4
SEED        = 42
NUM_WORKERS = 4
PRETRAINED_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth"

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ========== Helper functions ==========
def make_block_mask(P_len, mask_ratio=0.3, block_len=32):
    keep = np.ones(P_len, dtype=bool)
    target = int(P_len * mask_ratio)
    n_blocks = max(1, target // block_len)
    for _ in range(n_blocks):
        st = np.random.randint(0, max(1, P_len - block_len + 1))
        keep[st:st+block_len] = False
    while (~keep).sum() < target:
        keep[np.random.randint(0, P_len)] = False
    return keep

class HBNCBraModSSL(Dataset):
    """HBN EEG cache → (C,S,P), keep-mask"""
    def __init__(self, cache_dir, S=4, P=200):
        self.files = [os.path.join(cache_dir, f) for f in os.listdir(cache_dir) if f.endswith(".npy")]
        self.S, self.P = S, P
        self.win_len = S * P
    def __len__(self): return len(self.files)
    def __getitem__(self, i):
        arr = np.load(self.files[i], mmap_mode="r")
        seg = arr[np.random.randint(0, arr.shape[0])]
        if seg.shape[0] == 129: seg = seg[:-1,:]  # Cz 제거
        C,T = seg.shape
        if T < self.win_len:
            pad = np.zeros((C, self.win_len), np.float32); pad[:,:T]=seg; seg=pad
        else:
            st = np.random.randint(0, T - self.win_len + 1)
            seg = seg[:, st:st+self.win_len]
        seg = (seg - seg.mean(axis=1, keepdims=True)) / (seg.std(axis=1, keepdims=True)+1e-6)
        x = torch.tensor(seg.reshape(C, self.S, self.P), dtype=torch.float32)
        keep = np.stack([make_block_mask(self.P, MASK_RATIO, BLOCK_LEN) for _ in range(self.S)], axis=0)
        keep = torch.tensor(keep, dtype=torch.bool).unsqueeze(0)
        return x, keep

def masked_mse(pred, target, keep_bool, eps=1e-8):
    masked = (~keep_bool).float()
    diff2  = (pred - target)**2
    num = (diff2 * masked).sum()
    den = masked.sum() * pred.size(1) + eps
    return num/den

# ========== Main ==========
def main():
    ds = HBNCBraModSSL(CACHE_DIR, S, P)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True)

    # 1. Load pretrained CBraMod
    model = CBraMod().to(DEVICE)
    model.proj_out = nn.Identity()

    if os.path.exists(PRETRAINED_PATH):
        ckpt = torch.load(PRETRAINED_PATH, map_location="cpu")
        model.load_state_dict(ckpt, strict=False)
        print(f"✅ Loaded pretrained CBraMod weights from {PRETRAINED_PATH}")
    else:
        print("⚠️ Pretrained model not found, training from scratch.")

    # 2. Continue SSL with HBN EEG
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scaler = torch.cuda.amp.GradScaler(enabled=(DEVICE=="cuda"))

    for ep in range(1, EPOCHS+1):
        model.train(); total=0.0
        for x, keep in tqdm(loader, desc=f"CBraMod HBN-SSL Ep{ep}"):
            x, keep = x.to(DEVICE), keep.to(DEVICE)
            visible = x * keep.float()
            opt.zero_grad()
            with torch.cuda.amp.autocast(enabled=(DEVICE=="cuda")):
                pred = model(visible)
                loss = masked_mse(pred, x, keep)
            scaler.scale(loss).backward()
            scaler.step(opt); scaler.update()
            total += loss.item()
        print(f"[Ep{ep:02d}] masked MSE = {total/len(loader):.6f}")

    torch.save(model.state_dict(), "cbramod_ssl_hbn_from_pretrained.pth")
    print("✅ Saved fine-tuned SSL model: cbramod_ssl_hbn_from_pretrained.pth")

if __name__ == "__main__":
    main()


✅ Loaded pretrained CBraMod weights from /home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth


CBraMod HBN-SSL Ep1: 100%|██████████| 20/20 [00:02<00:00,  8.28it/s]


[Ep01] masked MSE = 0.970809


CBraMod HBN-SSL Ep2: 100%|██████████| 20/20 [00:02<00:00,  8.36it/s]


[Ep02] masked MSE = 0.844289


...

[Ep48] masked MSE = 0.513052


CBraMod HBN-SSL Ep49: 100%|██████████| 20/20 [00:02<00:00,  8.42it/s]


[Ep49] masked MSE = 0.518703


CBraMod HBN-SSL Ep50: 100%|██████████| 20/20 [00:02<00:00,  7.65it/s]


[Ep50] masked MSE = 0.514092
✅ Saved fine-tuned SSL model: cbramod_ssl_hbn_from_pretrained.pth


In [15]:
# finetune_hbn_regressor_from_ssl.py
import os, random, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from collections import defaultdict

# ---- CBraMod import ----
CBRAMOD_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod"
import sys
if CBRAMOD_PATH not in sys.path:
    sys.path.append(CBRAMOD_PATH)
from models.cbramod import CBraMod

# ---- Config ----
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5,17)]
SSL_CKPT  = "cbramod_ssl_hbn.pth"  # Stage1 output
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
S, P = 4, 200
N_CH = 128
K = 4
BATCH = 8
EPOCHS = 30
LR = 2e-4
WD = 1e-3
NUM_WORKERS = 4

# ---- Metadata load ----
def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs=[]
    for ds in datasets:
        pfile = os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df = pd.read_csv(pfile, sep="\t"); df["release"]=ds; dfs.append(df)
    df = pd.concat(dfs, ignore_index=True)
    df.columns=[c.lower() for c in df.columns]
    df=df.drop_duplicates(subset=["participant_id"]).set_index("participant_id")
    if "p_factor" not in df.columns:
        alt=[c for c in df.columns if "p" in c and "factor" in c]
        if alt: df=df.rename(columns={alt[0]:"p_factor"})
    for c in ["age","sex","ehq_total"]:
        if c not in df.columns: df[c]=np.nan
    return df

meta_df=load_all_participants()
labels_df=meta_df[["p_factor"]].dropna()
meta_extra=meta_df[["age","sex","ehq_total"]].copy()
meta_extra["sex"]=meta_extra["sex"].map({"M":1,"F":0,"m":1,"f":0}).fillna(0.5)
for col in ["age","ehq_total"]:
    m,s=meta_extra[col].mean(),meta_extra[col].std()+1e-6
    meta_extra[col]=(meta_extra[col]-m)/s

# ---- Indexing ----
def subj_from_fname(f): return f.split("_")[0]
def build_index(cache_dir, files, label_idx):
    idx=[]
    for f in files:
        subj=subj_from_fname(f)
        if subj not in label_idx: continue
        arr=np.load(os.path.join(cache_dir,f),mmap_mode="r")
        for s in range(arr.shape[0]): idx.append((f,s,subj))
    return idx

all_files=[f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
train_files=[f for f in all_files if "_train_" in f]
val_files=[f for f in all_files if "_val_" in f]
train_index=build_index(CACHE_DIR,train_files,labels_df.index)
val_index=build_index(CACHE_DIR,val_files,labels_df.index)

def subj_to_idx(idx):
    d=defaultdict(list)
    for i,(_,_,s) in enumerate(idx): d[s].append(i)
    return d
train_map=subj_to_idx(train_index); val_map=subj_to_idx(val_index)

# ---- Dataset ----
class WindowDS(Dataset):
    def __init__(self, cache_dir, index, labels, S=4,P=200,train=True):
        self.cache_dir=cache_dir; self.index=index; self.labels=labels
        self.S=S; self.P=P; self.W=S*P; self.train=train
    def __getitem__(self,i):
        f,s,subj=self.index[i]
        y=float(self.labels.loc[subj])
        x=np.load(os.path.join(self.cache_dir,f),mmap_mode="r")[s]
        if x.shape[0]==129: x=x[:-1,:]
        C,T=x.shape
        if T<self.W:
            pad=np.zeros((C,self.W),np.float32); pad[:,:T]=x; x=pad
        else:
            st=np.random.randint(0,T-self.W+1) if self.train else (T-self.W)//2
            x=x[:,st:st+self.W]
        x=(x-x.mean(1,keepdims=True))/(x.std(1,keepdims=True)+1e-6)
        return torch.tensor(x.reshape(C,self.S,self.P),dtype=torch.float32),torch.tensor(y),subj
    def __len__(self): return len(self.index)

class SubjectDS(Dataset):
    def __init__(self, base, subj_map, meta, K=4,train=True):
        self.base=base; self.map=subj_map; self.subjs=list(self.map.keys()); self.meta=meta; self.K=K; self.train=train
    def __getitem__(self,i):
        subj=self.subjs[i]; idxs=self.map[subj]
        sel=random.choices(idxs,k=self.K)
        Xs,ys=[],[]
        for j in sel:
            x,y,s=self.base[j]; Xs.append(x); ys.append(y)
        X=torch.stack(Xs,0); meta_vec=torch.tensor(self.meta.loc[subj].values,dtype=torch.float32)
        return X,ys[0],meta_vec,subj
    def __len__(self): return len(self.subjs)

# ---- Model ----
class CBraModRegressor(nn.Module):
    def __init__(self, meta_dim=3):
        super().__init__()
        self.backbone=CBraMod(); self.backbone.proj_out=nn.Identity()
        self.meta_fc=nn.Sequential(nn.Linear(meta_dim,32),nn.ReLU())
        self.regressor=nn.Sequential(nn.Linear(N_CH+32,128),nn.ReLU(),nn.Linear(128,1))
    def forward(self,X,meta):
        B,K=X.shape[:2]
        X=X.view(B*K,N_CH,S,P)
        F=self.backbone(X)
        F=F.mean((-1,-2))        # GAP
        F=F.view(B,K,N_CH).mean(1)
        M=self.meta_fc(meta)
        return self.regressor(torch.cat([F,M],1)).squeeze(-1)

# ---- Train/Eval ----
def evaluate(model, loader, mean, std):
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for X, y, meta, _ in loader:
            X, meta = X.to(DEVICE), meta.to(DEVICE)
            y = y.to(torch.float32).to(DEVICE)

            p = model(X, meta)                   # (B,)
            # 🔍 유효값만 사용 (치환 X)
            mask = torch.isfinite(p) & torch.isfinite(y)
            if mask.sum() == 0:
                continue

            p = p[mask]
            y = y[mask]

            p_denorm = p * std + mean
            preds.extend(p_denorm.cpu().numpy())
            trues.extend(y.cpu().numpy())

    preds, trues = np.asarray(preds), np.asarray(trues)
    if len(trues) == 0:
        return np.nan, np.nan, np.nan

    # 붕괴 진단 로그(한 번만 보는 용도)
    # print("pred mean/std:", preds.mean(), preds.std())

    r2 = 0.0 if np.var(trues) < 1e-8 else r2_score(trues, preds)
    return mean_absolute_error(trues, preds), mean_squared_error(trues, preds), r2


def main():
    tr_base=WindowDS(CACHE_DIR,train_index,labels_df["p_factor"],S,P,train=True)
    va_base=WindowDS(CACHE_DIR,val_index,labels_df["p_factor"],S,P,train=False)
    tr_ds=SubjectDS(tr_base,train_map,meta_extra,K,train=True)
    va_ds=SubjectDS(va_base,val_map,meta_extra,K,train=False)
    tr_loader=DataLoader(tr_ds,batch_size=BATCH,shuffle=True,num_workers=NUM_WORKERS)
    va_loader=DataLoader(va_ds,batch_size=BATCH,shuffle=False,num_workers=NUM_WORKERS)

    model = CBraModRegressor().to(DEVICE)
    if os.path.exists(SSL_CKPT):
        ckpt = torch.load(SSL_CKPT, map_location="cpu")
        model.backbone.load_state_dict(ckpt, strict=False)
        print("✅ Loaded SSL backbone weights.")
    else:
        print("⚠️ No SSL checkpoint found. Training from scratch.")

    # 🔓 혹시라도 freeze 되어 있으면 전부 풀기
    for p in model.parameters():
        p.requires_grad = False

    # 🧪 진짜로 풀렸는지 한 번만 로깅
    n_trainable = sum(p.requires_grad for p in model.parameters())
    print(f"Trainable params tensors: {n_trainable}")

    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

    mean,std=labels_df["p_factor"].mean(),labels_df["p_factor"].std()+1e-6
    best={"mae":1e9,"mse":1e9,"r2":-1}
    for ep in range(1,EPOCHS+1):
        model.train()
        for X, y, m, subj in tqdm(tr_loader, desc=f"Train Ep{ep}"):
            X, m = X.to(DEVICE), m.to(DEVICE)
            y = y.to(torch.float32).to(DEVICE)
            y_n = ((y - mean) / std)

            opt.zero_grad()
            p = model(X, m)  # (B,)

            # 🔍 수치 진단
            nan_frac = torch.isnan(p).float().mean().item()
            if nan_frac > 0:
                # 이 배치 스킵 (덮어쓰기 대신 버리기)
                # 필요하면 여기서 gradient clipping/ LR 낮추기 등을 고려
                continue

            loss = 0.5 * (F.l1_loss(p, y_n) + F.mse_loss(p, y_n))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # 폭주 방지
            opt.step()

        val_mae,val_mse,val_r2=evaluate(model,va_loader,mean,std)
        print(f"[Ep{ep:02d}] Val MAE={val_mae:.4f} MSE={val_mse:.4f} R²={val_r2:.3f}")
        if val_mae<best["mae"]:
            best={"mae":val_mae,"mse":val_mse,"r2":val_r2}
            torch.save(model.state_dict(),"cbramod_hbn_regressor_from_ssl_best.pth")
            print(" ✓ Saved best model")

if __name__=="__main__":
    main()


✅ Loaded SSL backbone weights.
Trainable params tensors: 0


Train Ep1:   0%|          | 0/38 [00:02<?, ?it/s]


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [16]:
# domain_adapt_regression_cbramod_adversarial.py
import os, sys, random, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from collections import defaultdict
from itertools import cycle

# ------------------------------------------------------------
# Config
# ------------------------------------------------------------
CBRAMOD_PATH = "/home/RA/EEG_Challenge/Challenge2/CBraMod"
if CBRAMOD_PATH not in sys.path:
    sys.path.append(CBRAMOD_PATH)
from models.cbramod import CBraMod  # repo의 모델

CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
DATASETS  = [f"ds0055{i:02d}" for i in range(5, 17)]
SSL_CKPT  = "/home/RA/EEG_Challenge/Challenge2/CBraMod/pretrained_weights/pretrained_weights.pth"  # 사전학습 백본 경로

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
S, P    = 4, 200
WIN_T   = S * P
N_CH    = 128          # 129면 Cz 제거
BATCH   = 8
EPOCHS  = 30
LR      = 2e-4
WD      = 1e-3
NUM_WORKERS = 4
LAMBDA_DA   = 0.2      # domain loss 가중치
SEED    = 42

def set_seed(seed=SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
set_seed()

# ------------------------------------------------------------
# Utilities
# ------------------------------------------------------------
def subj_from_fname(f): return f.split("_")[0]

def load_all_participants(bids_root=BIDS_ROOT, datasets=DATASETS):
    dfs=[]
    for ds in datasets:
        pfile=os.path.join(bids_root, ds, "participants.tsv")
        if os.path.exists(pfile):
            df=pd.read_csv(pfile, sep="\t"); df["release"]=ds; dfs.append(df)
    df=pd.concat(dfs, ignore_index=True)
    df.columns=[c.lower() for c in df.columns]
    df=df.drop_duplicates(subset=["participant_id"]).set_index("participant_id")
    if "p_factor" not in df.columns:
        alt=[c for c in df.columns if "p" in c and "factor" in c]
        if alt: df=df.rename(columns={alt[0]:"p_factor"})
    for c in ["age","sex","ehq_total"]:
        if c not in df.columns: df[c]=np.nan
    return df

meta_df   = load_all_participants()
labels_df = meta_df[["p_factor"]].dropna()

# ------------------------------------------------------------
# Indexing
# ------------------------------------------------------------
def build_index(cache_dir, files, label_idx):
    idx=[]
    for f in files:
        subj=subj_from_fname(f)
        if subj not in label_idx: 
            continue
        arr=np.load(os.path.join(cache_dir,f), mmap_mode="r")
        for s in range(arr.shape[0]):
            idx.append((f, s, subj))
    return idx

all_files = [f for f in os.listdir(CACHE_DIR) if f.endswith(".npy")]
src_files = [f for f in all_files if "_train_" in f]  # source domain (labeled)
tgt_files = [f for f in all_files if "_val_" in f]    # target domain

src_index = build_index(CACHE_DIR, src_files, labels_df.index)
tgt_index = build_index(CACHE_DIR, tgt_files, labels_df.index)

# ------------------------------------------------------------
# Datasets
# ------------------------------------------------------------
class WindowDS(Dataset):
    """(segments, C, T) 캐시 → (C, S, P), y"""
    def __init__(self, cache_dir, index, labels, S=4, P=200, train=True):
        self.cache_dir=cache_dir; self.index=index; self.labels=labels
        self.S=S; self.P=P; self.W=S*P; self.train=train

    def __len__(self): return len(self.index)

    def __getitem__(self, i):
        f, s, subj = self.index[i]
        y = float(self.labels.loc[subj])
        x = np.load(os.path.join(self.cache_dir, f), mmap_mode="r")[s]  # (C, T)

        # Cz 제거 (129 -> 128)
        if x.shape[0] == 129: 
            x = x[:-1, :]

        C, T = x.shape
        # pad/crop to W
        if T < self.W:
            pad = np.zeros((C, self.W), np.float32)
            pad[:, :T] = x; x = pad
        else:
            st = np.random.randint(0, T - self.W + 1) if self.train else (T - self.W)//2
            x = x[:, st:st+self.W]

        # per-channel z-score
        x = (x - x.mean(axis=1, keepdims=True)) / (x.std(axis=1, keepdims=True) + 1e-6)

        return torch.tensor(x.reshape(C, self.S, self.P), dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

src_ds = WindowDS(CACHE_DIR, src_index, labels_df["p_factor"], S, P, train=True)
tgt_ds = WindowDS(CACHE_DIR, tgt_index, labels_df["p_factor"], S, P, train=False)

src_loader = DataLoader(src_ds, batch_size=BATCH, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
tgt_loader = DataLoader(tgt_ds, batch_size=BATCH, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
tgt_eval_loader = DataLoader(tgt_ds, batch_size=BATCH, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

# ------------------------------------------------------------
# Model: CBraMod backbone + GRL domain head + Regressor
# ------------------------------------------------------------
class GradientReversalFn(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lamb): 
        ctx.lamb = lamb
        return x.view_as(x)
    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lamb * grad_output, None

class GRL(nn.Module):
    def __init__(self, lamb=1.0): super().__init__(); self.lamb=float(lamb)
    def forward(self, x, lamb=None):
        if lamb is None: lamb = self.lamb
        return GradientReversalFn.apply(x, float(lamb))

class DomainAdaptRegressor(nn.Module):
    def __init__(self, meta_dim=0):  # meta 안 쓰면 0
        super().__init__()
        self.backbone = CBraMod()
        self.backbone.proj_out = nn.Identity()      # (B, C, S, P)
        self.grl = GRL()
        self.domain_head = nn.Sequential(
            nn.Linear(N_CH, 64), nn.ReLU(),
            nn.Linear(64, 2)                         # source / target
        )
        # 간단 회귀헤드: GAP 후 바로 회귀
        self.regressor = nn.Sequential(
            nn.Linear(N_CH + meta_dim, 128), nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, x, lamb_da=0.0, meta=None):
        f = self.backbone(x)             # (B, C, S, P)
        f = f.mean(dim=(-1, -2))         # (B, C) = (B, 128)
        # regression
        if meta is None:
            z = f
        else:
            z = torch.cat([f, meta], dim=1)
        y_hat = self.regressor(z).squeeze(-1)
        # domain logits
        d_logits = self.domain_head(self.grl(f, lamb_da))
        return y_hat, d_logits

# ------------------------------------------------------------
# Metrics
# ------------------------------------------------------------
def safe_r2(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    if y_true.size < 2 or np.var(y_true) < 1e-8:
        return 0.0
    return r2_score(y_true, y_pred)

@torch.no_grad()
def evaluate(model, loader, y_mean, y_std):
    model.eval()
    preds, trues = [], []
    for x, y in loader:
        x = x.to(DEVICE); y = y.to(torch.float32).to(DEVICE)
        p, _ = model(x, lamb_da=0.0, meta=None)     # eval: 도메인 로짓 무시
        # 역정규화
        p = p * y_std + y_mean
        # 유효값만
        mask = torch.isfinite(p) & torch.isfinite(y)
        if mask.sum() == 0: 
            continue
        preds.extend(p[mask].cpu().numpy())
        trues.extend(y[mask].cpu().numpy())
    if len(trues) == 0:
        return np.nan, np.nan, np.nan
    preds, trues = np.array(preds), np.array(trues)
    return mean_absolute_error(trues, preds), mean_squared_error(trues, preds), safe_r2(trues, preds)

# ------------------------------------------------------------
# Train
# ------------------------------------------------------------
def main():
    model = DomainAdaptRegressor(meta_dim=0).to(DEVICE)
    if os.path.exists(SSL_CKPT):
        ckpt = torch.load(SSL_CKPT, map_location="cpu")
        model.backbone.load_state_dict(ckpt, strict=False)
        print("✅ Loaded SSL pretrained CBraMod backbone.")
    else:
        print("⚠️ No SSL checkpoint found. Training backbone from scratch.")

    # 필요하면 backbone 일부만 풀 수도 있음. 여기선 전체 미세조정 허용
    for p in model.backbone.parameters():
        p.requires_grad = True

    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    ce  = nn.CrossEntropyLoss()

    y_mean = labels_df["p_factor"].mean()
    y_std  = labels_df["p_factor"].std() + 1e-6

    best = {"mae": np.inf, "mse": np.inf, "r2": -1.0}

    # target loader가 더 짧으면 cycle로 반복
    tgt_iter = cycle(tgt_loader)

    for ep in range(1, EPOCHS+1):
        model.train()
        total = 0.0
        for x_s, y_s in tqdm(src_loader, desc=f"[Train DA] Ep{ep}"):
            x_t, _ = next(tgt_iter)

            x_s = x_s.to(DEVICE); y_s = y_s.to(torch.float32).to(DEVICE)
            x_t = x_t.to(DEVICE)

            # label 정규화
            y_s_n = (y_s - y_mean) / y_std

            opt.zero_grad()

            # source: regression + domain(0)
            p_s, d_s = model(x_s, lamb_da=LAMBDA_DA, meta=None)
            loss_reg = 0.5 * (F.l1_loss(p_s, y_s_n) + F.mse_loss(p_s, y_s_n))
            loss_dom_s = ce(d_s, torch.zeros(len(x_s), dtype=torch.long, device=DEVICE))

            # target: domain(1)만
            _, d_t = model(x_t, lamb_da=LAMBDA_DA, meta=None)
            loss_dom_t = ce(d_t, torch.ones(len(x_t), dtype=torch.long, device=DEVICE))

            loss = loss_reg + LAMBDA_DA * (loss_dom_s + loss_dom_t)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            opt.step()
            total += loss.item()

        # ---- Evaluation on target domain (with labels) ----
        val_mae, val_mse, val_r2 = evaluate(model, tgt_eval_loader, y_mean, y_std)
        print(f"[Ep{ep:02d}] TrainLoss={total/max(1,len(src_loader)):.4f} | "
              f"TGT Val MAE={val_mae:.4f} MSE={val_mse:.4f} R²={val_r2:.3f}")

        if val_mae < best["mae"]:
            best = {"mae": val_mae, "mse": val_mse, "r2": val_r2}
            torch.save(model.state_dict(), "cbramod_da_regressor_best.pth")
            print("✓ Saved best: cbramod_da_regressor_best.pth")

    print(f"\nBest Target-domain Val — MAE={best['mae']:.4f} | MSE={best['mse']:.4f} | R²={best['r2']:.3f}")

if __name__ == "__main__":
    main()


✅ Loaded SSL pretrained CBraMod backbone.


[Train DA] Ep1: 100%|██████████| 12430/12430 [31:37<00:00,  6.55it/s] 


[Ep01] TrainLoss=0.6597 | TGT Val MAE=0.9641 MSE=1.2424 R²=-0.174
✓ Saved best: cbramod_da_regressor_best.pth


[Train DA] Ep2: 100%|██████████| 12430/12430 [31:29<00:00,  6.58it/s]


[Ep02] TrainLoss=0.3646 | TGT Val MAE=1.0240 MSE=1.3978 R²=-0.321


...

[Ep28] TrainLoss=0.1238 | TGT Val MAE=0.8714 MSE=1.0084 R²=0.047
✓ Saved best: cbramod_da_regressor_best.pth


[Train DA] Ep29: 100%|██████████| 12430/12430 [30:36<00:00,  6.77it/s]


[Ep29] TrainLoss=0.1216 | TGT Val MAE=0.9081 MSE=1.0893 R²=-0.030


[Train DA] Ep30: 100%|██████████| 12430/12430 [30:40<00:00,  6.75it/s]


[Ep30] TrainLoss=0.1203 | TGT Val MAE=0.9077 MSE=1.1073 R²=-0.047

Best Target-domain Val — MAE=0.8714 | MSE=1.0084 | R²=0.047


## Self-supervised learning raw data

In [1]:
# ssl_pretrain_raw_eegconformer_from_cache_and_missing_v2.py
import os, random, numpy as np, torch, mne, warnings
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from braindecode.models import EEGConformer

# ---------------- CONFIG ----------------
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
BIDS_ROOT = "/data5/open_data/HBN/EEG_BIDS"
MISSING_TXT = "/home/RA/EEG_Challenge/Challenge2/logs/missing_cache_files.txt"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
SFREQ = 100
N_CHANS = 128
CROP_T = 1000
MASK_RATIO = 0.3
BLOCK_LEN = 100

BATCH_SIZE = 8
EPOCHS = 50
LR = 3e-4
WD = 1e-4
SEED = 42

warnings.filterwarnings("ignore")
mne.set_log_level("CRITICAL")
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)

# ============================================================
# 1. 파일 리스트 구성 (cache + missing 텍스트 기반)
# ============================================================
def build_eeg_filelist(cache_dir, missing_txt, bids_root):
    cache_files = [f for f in os.listdir(cache_dir) if f.endswith(".npy")]
    found_from_cache, not_found = [], []

    # -----------------------------
    # ✅ 1) CACHE FILES 처리
    # -----------------------------
    for f in cache_files:
        subj = f.split("_")[0]  # sub-XXXX
        task = f.split("_task-")[1].split("_")[0]
        run = None
        if "_run-" in f:
            run = f.split("_run-")[1].split("_")[0]
        matched = False

        for ds in os.listdir(bids_root):
            eeg_dir = os.path.join(bids_root, ds, subj, "eeg")
            if not os.path.exists(eeg_dir):
                continue

            for fname in os.listdir(eeg_dir):
                if not fname.endswith(".set"):
                    continue
                # run 없는 경우
                if run is None:
                    if f"task-{task}" in fname:
                        found_from_cache.append(os.path.join(eeg_dir, fname))
                        matched = True
                        break
                else:
                    # run 있는 경우: 1 / 01 패딩 모두 허용
                    run_variants = [f"run-{run}", f"run-0{run}"]
                    if f"task-{task}" in fname and any(r in fname for r in run_variants):
                        found_from_cache.append(os.path.join(eeg_dir, fname))
                        matched = True
                        break
            if matched:
                break
        if not matched:
            not_found.append(f)

    # -----------------------------
    # ✅ 2) MISSING FILES 처리
    # -----------------------------
    found_from_missing = []
    if os.path.exists(missing_txt):
        with open(missing_txt, "r") as f:
            for line in f:
                base = line.strip()
                if len(base) == 0:
                    continue
                subj = base.split("_task-")[0]
                task = base.split("_task-")[1].split("_")[0]
                run = None
                if "_run-" in base:
                    run = base.split("_run-")[1].split("_")[0]

                for ds in os.listdir(bids_root):
                    eeg_dir = os.path.join(bids_root, ds, subj, "eeg")
                    if not os.path.exists(eeg_dir):
                        continue

                    for fname in os.listdir(eeg_dir):
                        if not fname.endswith(".set"):
                            continue
                        if run is None:
                            if f"task-{task}" in fname:
                                found_from_missing.append(os.path.join(eeg_dir, fname))
                                break
                        else:
                            run_variants = [f"run-{run}", f"run-0{run}"]
                            if f"task-{task}" in fname and any(r in fname for r in run_variants):
                                found_from_missing.append(os.path.join(eeg_dir, fname))
                                break

    # -----------------------------
    # ✅ 3) 최종 결과
    # -----------------------------
    all_files = sorted(list(set(found_from_cache + found_from_missing)))
    print(f"✅ Total EEG files: {len(all_files)} "
          f"(cache={len(found_from_cache)}, missing={len(found_from_missing)})")
    if len(not_found) > 0:
        print(f"⚠️ {len(not_found)} cache-based files not matched to .set:")
        print("\n".join(not_found[:15]), "...")
    return all_files


# ============================================================
# 2. Dataset
# ============================================================
def make_block_mask(T, mask_ratio=0.3, block_len=50):
    mask = np.ones(T, dtype=bool)
    n_mask = int(T * mask_ratio)
    n_blocks = max(1, n_mask // block_len)
    for _ in range(n_blocks):
        st = np.random.randint(0, T - block_len)
        mask[st:st + block_len] = False
    return mask

class RawEEGConformerSSL(Dataset):
    def __init__(self, file_list, crop_len=1000, mask_ratio=0.3):
        self.files = file_list
        self.crop_len = crop_len
        self.mask_ratio = mask_ratio

    def __len__(self): return len(self.files)

    def __getitem__(self, i):
        path = self.files[i]
        raw = mne.io.read_raw_eeglab(path, preload=True, verbose=False)
        raw.filter(0.5, 50., fir_design="firwin")
        raw.set_eeg_reference("average", projection=False)
        raw.resample(SFREQ)

        x = raw.get_data()
        if x.shape[0] == 129:
            x = x[:-1, :]  # Cz 제거
        x = (x - x.mean(axis=1, keepdims=True)) / (x.std(axis=1, keepdims=True) + 1e-6)

        T = x.shape[1]
        if T < self.crop_len:
            pad = np.zeros((x.shape[0], self.crop_len))
            pad[:, :T] = x
            x = pad
        else:
            st = np.random.randint(0, T - self.crop_len + 1)
            x = x[:, st:st + self.crop_len]

        mask = make_block_mask(self.crop_len, self.mask_ratio, BLOCK_LEN)
        keep = torch.tensor(mask, dtype=torch.bool).unsqueeze(0)
        return torch.tensor(x, dtype=torch.float32), keep

# ============================================================
# 3. EEG-Conformer Masked Autoencoder
# ============================================================
class EEGMaskedAutoencoder(nn.Module):
    def __init__(self, n_chans=128, n_times=1000, sfreq=100, embed_dim=256):
        super().__init__()
        self.encoder = EEGConformer(
            n_chans=n_chans, n_outputs=embed_dim,
            n_times=n_times, sfreq=sfreq
        )
        self.decoder = nn.Sequential(
            nn.Conv1d(embed_dim, embed_dim, 3, padding=1),
            nn.GELU(),
            nn.Conv1d(embed_dim, n_chans, 1)
        )

    def forward(self, x, keep):
        visible = x * keep.float()
        z = self.encoder(visible)
        if z.ndim == 2:
            z = z.unsqueeze(-1)
        if z.shape[-1] != x.shape[-1]:
            z = F.interpolate(z, size=x.shape[-1], mode="linear", align_corners=False)
        return self.decoder(z)

def masked_mse(pred, target, keep, eps=1e-8):
    mask = (~keep).float()
    diff2 = (pred - target) ** 2
    num = (diff2 * mask).sum()
    den = mask.sum() * pred.size(1) + eps
    return num / den

# ============================================================
# 4. Train
# ============================================================
def main():
    file_list = build_eeg_filelist(CACHE_DIR, MISSING_TXT, BIDS_ROOT)
    ds = RawEEGConformerSSL(file_list, crop_len=CROP_T, mask_ratio=MASK_RATIO)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)

    model = EEGMaskedAutoencoder(N_CHANS, CROP_T, SFREQ, 256).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)

    for ep in range(1, EPOCHS + 1):
        model.train(); total = 0.0
        for x, keep in tqdm(loader, desc=f"EEGConformer SSL Ep{ep}"):
            x, keep = x.to(DEVICE), keep.to(DEVICE)
            opt.zero_grad()
            x_hat = model(x, keep)
            loss = masked_mse(x_hat, x, keep)
            loss.backward(); opt.step()
            total += loss.item()
        print(f"[Ep{ep:02d}] masked MSE={total/len(loader):.6f}")

    torch.save(model.encoder.state_dict(), "eegconformer_ssl_raw_from_cache_missing.pth")
    print("✅ Saved pretrained encoder -> eegconformer_ssl_raw_from_cache_missing.pth")

if __name__ == "__main__":
    main()


/usr/lib/python3/dist-packages/jinja2/loaders.py:222: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DefaultProvider, ResourceManager, \


✅ Total EEG files: 340 (cache=318, missing=23)


EEGConformer SSL Ep1: 100%|██████████| 43/43 [42:00<00:00, 58.62s/it]  


[Ep01] masked MSE=0.601568


EEGConformer SSL Ep2: 100%|██████████| 43/43 [39:28<00:00, 55.09s/it]  


...

[Ep17] masked MSE=0.581924


EEGConformer SSL Ep18: 100%|██████████| 43/43 [40:24<00:00, 56.38s/it]  


[Ep18] masked MSE=1.073441


EEGConformer SSL Ep19:  19%|█▊        | 8/43 [09:14<40:25, 69.31s/it]   


KeyboardInterrupt: 

In [4]:
# ssl_pretrain_eegconformer_fast.py
import os, random, numpy as np, torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from braindecode.models import EEGConformer

# ---------------- Config ----------------
CACHE_DIR = "/data5/open_data/HBN/cache_eeg_windows"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ==== SPEED-OPTIMIZED PARAMETERS ====
SFREQ = 100
N_CHANS_TOTAL = 129
N_CHANS_USED = 64          # 랜덤 샘플링할 채널 수
CROP_T = 400               # crop 단축
MASK_RATIO = 0.3
BATCH_SIZE = 16
EPOCHS = 30
LR = 3e-4
WD = 1e-4
SEED = 42
NUM_WORKERS = 8
PREFETCH = 4
ACCUM_STEPS = 2            # gradient accumulation (가상 batch = BATCH_SIZE*ACCUM_STEPS)

torch.manual_seed(SEED); np.random.seed(SEED); random.seed(SEED)
torch.backends.cudnn.benchmark = True

# ============================================================
# Dataset: cache → masked reconstruction
# ============================================================
def make_block_mask(T, mask_ratio=0.3, block_len=40):
    mask = np.ones(T, dtype=bool)
    n_mask = int(T * mask_ratio)
    n_blocks = max(1, n_mask // block_len)
    for _ in range(n_blocks):
        start = np.random.randint(0, T - block_len)
        mask[start:start + block_len] = False
    return mask

class MaskedEEGDataset(Dataset):
    def __init__(self, cache_dir, crop_len=400, mask_ratio=0.3, n_chans_used=64):
        self.files = [os.path.join(cache_dir, f) for f in os.listdir(cache_dir) if f.endswith(".npy")]
        self.crop_len = crop_len
        self.mask_ratio = mask_ratio
        self.n_chans_used = n_chans_used

    def __len__(self): return len(self.files)

    def __getitem__(self, i):
        arr = np.load(self.files[i], mmap_mode="r")          # (segments, C, T)
        seg = arr[random.randint(0, arr.shape[0]-1)]         # (C, T)
        if seg.shape[0] == N_CHANS_TOTAL:                    # Cz 제거
            seg = seg[:-1, :]

        # 랜덤 채널 샘플링 (128→64)
        if seg.shape[0] > self.n_chans_used:
            sel = np.random.choice(seg.shape[0], self.n_chans_used, replace=False)
            seg = seg[sel, :]

        C, T = seg.shape
        if T < self.crop_len:
            pad = np.zeros((C, self.crop_len), np.float32)
            pad[:, :T] = seg
            seg = pad
        else:
            start = np.random.randint(0, T - self.crop_len + 1)
            seg = seg[:, start:start+self.crop_len]

        # normalization
        seg = (seg - seg.mean(axis=1, keepdims=True)) / (seg.std(axis=1, keepdims=True) + 1e-6)
        x = torch.tensor(seg, dtype=torch.float32)           # (C, T)

        keep = make_block_mask(self.crop_len, self.mask_ratio)
        keep = torch.tensor(keep, dtype=torch.bool).unsqueeze(0)  # (1, T)
        return x, keep

# ============================================================
# Model: EEGConformer encoder + 2-layer decoder
# ============================================================
class EEGMaskedAutoencoder(nn.Module):
    def __init__(self, n_chans, n_times, sfreq, embed_dim=256):
        super().__init__()
        self.encoder = EEGConformer(n_chans=n_chans, n_outputs=embed_dim, n_times=n_times, sfreq=sfreq)
        self.decoder = nn.Sequential(
            nn.Conv1d(embed_dim, embed_dim//2, kernel_size=3, padding=1),
            nn.GELU(),
            nn.Conv1d(embed_dim//2, n_chans, kernel_size=1)
        )

    def forward(self, x, keep):
        B, C, T = x.shape
        visible = x * keep.float()
        z = self.encoder(visible)
        if z.ndim == 2:
            z = z.unsqueeze(-1)
        if z.shape[-1] != T:
            z = F.interpolate(z, size=T, mode="linear", align_corners=False)
        return self.decoder(z)

def masked_mse(pred, target, keep, eps=1e-8):
    masked = (~keep).float()
    diff2 = (pred - target) ** 2
    num = (diff2 * masked).sum()
    den = masked.sum() * pred.size(1) + eps
    return num / den

# ============================================================
# Train Loop (with AMP + Grad Accum)
# ============================================================
def main():
    ds = MaskedEEGDataset(CACHE_DIR, crop_len=CROP_T, mask_ratio=MASK_RATIO, n_chans_used=N_CHANS_USED)
    loader = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=NUM_WORKERS, prefetch_factor=PREFETCH,
                        pin_memory=True, persistent_workers=True, drop_last=True)

    model = EEGMaskedAutoencoder(N_CHANS_USED, CROP_T, SFREQ).to(DEVICE)
    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=WD)
    scaler = torch.cuda.amp.GradScaler(enabled=True)

    for ep in range(1, EPOCHS + 1):
        model.train(); running = 0.0
        opt.zero_grad(set_to_none=True)
        for step, (x, keep) in enumerate(tqdm(loader, desc=f"SSL Ep{ep}")):
            x = x.to(DEVICE, non_blocking=True)
            keep = keep.to(DEVICE, non_blocking=True)

            with torch.cuda.amp.autocast():
                x_hat = model(x, keep)
                loss = masked_mse(x_hat, x, keep) / ACCUM_STEPS

            scaler.scale(loss).backward()
            if (step + 1) % ACCUM_STEPS == 0:
                scaler.step(opt); scaler.update(); opt.zero_grad(set_to_none=True)

            running += loss.item() * ACCUM_STEPS  # 보정

        print(f"[Ep {ep:02d}] masked-recon MSE = {running / len(loader):.5f}")

    torch.save(model.encoder.state_dict(), f"pretrained_eegconformer_fast_{N_CHANS_USED}ch_crop{CROP_T}.pth")
    print(f"✅ Saved: pretrained_eegconformer_fast_{N_CHANS_USED}ch_crop{CROP_T}.pth")

if __name__ == "__main__":
    main()


SSL Ep1: 100%|██████████| 19/19 [00:10<00:00,  1.81it/s]


[Ep 01] masked-recon MSE = 0.99343


SSL Ep2: 100%|██████████| 19/19 [00:04<00:00,  4.12it/s]


[Ep 02] masked-recon MSE = 0.98232


SSL Ep3: 100%|██████████| 19/19 [00:09<00:00,  2.05it/s]


...

[Ep 28] masked-recon MSE = 0.95642


SSL Ep29: 100%|██████████| 19/19 [00:03<00:00,  5.17it/s]


[Ep 29] masked-recon MSE = 0.95923


SSL Ep30: 100%|██████████| 19/19 [00:03<00:00,  5.36it/s]


[Ep 30] masked-recon MSE = 0.94153
✅ Saved: pretrained_eegconformer_fast_64ch_crop400.pth
